## 🧪 E5: 데이터 증강 + 가중치 반영 실험

> **목표**: E4 기반 + 실제 데이터 비율(정상:악성 = 4:1) 반영 + 특수문자 정상 강화

### 배경
- E4에서 LOL 욕설 + 특수문자 정상 데이터 증강으로 성능 향상
- 실제 운영 환경: 정상:악성 ≈ 4:1
- 특수문자(`@`, `^`, `*` 등) 오탐 문제 해결 필요

### 실험 내용
1. **특수문자 정상 데이터 추가 강화** → FP 감소
2. **class_weight 적용** (w0=4, w1=1) → 실제 비율 반영

### 변경 사항
- 데이터: E4 + 특수문자 정상 추가
- `class_weights`: **[4, 1]** (정상 4배 중시)
- `metric_for_best_model`: f1

In [ ]:
# ========================================
# 📦 패키지 설치 (필요시)
# ========================================
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q
%pip install transformers==4.42.0 datasets accelerate scikit-learn pandas matplotlib seaborn -q

In [ ]:
# ========================================
# 🔧 경로 설정
# ========================================
import os

DATA_DIR = r"c:\00_Sookmyunguniv\DACOS(2025)\2학기\프로젝트\github\25-2-team3\data"
OUTPUT_DIR = r"./E5_output"

# 데이터 파일 경로
MAIN_DATA = os.path.join(DATA_DIR, "combined_abusive_shuffled_20k.csv")
NORMAL_DATA = os.path.join(DATA_DIR, "nonabusive_merged_shuffled_sample20000.csv")
LOL_DATA = os.path.join(DATA_DIR, "LOL_badwords_all_merged.csv")

if not os.path.exists(DATA_DIR):
    print(f"⚠️ 데이터 폴더를 찾을 수 없습니다: {DATA_DIR}")
else:
    print(f"📂 데이터 폴더: {DATA_DIR}")
print(f"📁 출력 폴더: {OUTPUT_DIR}")

In [ ]:
# ========================================
# ⭐ 실험 파라미터 설정
# ========================================

# 클래스 가중치 설정 (정상:악성 = 4:1 반영)
# 실제 환경에서 정상이 4배 많으므로, 정상 오분류(FP)에 더 큰 벌점
W0, W1 = 4.0, 1.0  # 정상 4배 중시 → FP 감소

print(f"⚖️ Class weights: [w0={W0}, w1={W1}]")
print(f"   → 정상 {W0/W1:.0f}배 중시 (FP 감소, 실제 비율 반영)")

In [ ]:
# ========================================
# 📚 라이브러리 임포트
# ========================================
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_auc_score
)
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import matplotlib.pyplot as plt
import seaborn as sns

# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
if torch.cuda.is_available():
    print(f"✅ GPU 사용 가능: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPU 사용 불가, CPU로 실행")

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# ========================================
# 📊 1) 기존 데이터 로드
# ========================================
df_main = pd.read_csv(MAIN_DATA)
df_normal = pd.read_csv(NORMAL_DATA)

# 기존 데이터 합치기 (text, label 컬럼 통일)
if 'text' not in df_main.columns:
    df_main = df_main.rename(columns={df_main.columns[0]: 'text'})
if 'label' not in df_main.columns:
    df_main['label'] = 1  # 악성

if 'text' not in df_normal.columns:
    df_normal = df_normal.rename(columns={df_normal.columns[0]: 'text'})
if 'label' not in df_normal.columns:
    df_normal['label'] = 0  # 정상

df_base = pd.concat([df_main[['text', 'label']], df_normal[['text', 'label']]], ignore_index=True)
print(f"📊 기존 데이터: {len(df_base)}개")
print(f"   - 정상: {len(df_base[df_base['label']==0])}개")
print(f"   - 악성: {len(df_base[df_base['label']==1])}개")

In [ ]:
# ========================================
# 🎮 2) LOL 욕설 데이터 추가
# ========================================
df_lol = pd.read_csv(LOL_DATA)
print(f"📊 LOL 욕설 원본: {len(df_lol)}개 행")

# origin(원본 욕설) + variant(변형 욕설) 모두 사용
lol_origins = df_lol['origin'].dropna().unique().tolist()
lol_variants = df_lol['variant'].dropna().unique().tolist()

# 합쳐서 중복 제거
all_lol_words = list(set(lol_origins + lol_variants))
print(f"   - origin(원본): {len(lol_origins)}개")
print(f"   - variant(변형): {len(lol_variants)}개")
print(f"   - 합계(중복제거): {len(all_lol_words)}개")

df_lol_aug = pd.DataFrame({
    'text': all_lol_words,
    'label': 1  # 악성
})

# 기존 데이터와 중복 제거
existing_texts = set(df_base['text'].astype(str).str.lower())
df_lol_aug = df_lol_aug[~df_lol_aug['text'].astype(str).str.lower().isin(existing_texts)]

print(f"✅ LOL 욕설 추가: {len(df_lol_aug)}개 (기존 데이터와 중복 제거 후)")

In [ ]:
# ========================================
# 🔣 3) 특수문자 정상 데이터 생성 (강화 버전)
# ========================================
# E4보다 더 많은 특수문자 패턴 추가
special_char_samples = [
    # 단일 특수문자
    "@", "#", "$", "%", "^", "&", "*", "!", "?", "~",
    ".", ",", ";", ":", "'", "\"", "`",
    "(", ")", "[", "]", "{", "}", "<", ">",
    "/", "\\", "|", "-", "_", "=", "+",
    
    # 이모티콘/표정 (강화)
    "^^", "^0^", "^_^", "^^;", "^o^", "^^7", "^-^", "^ㅁ^", "^ㅡ^",
    ":)", ":(", ":D", ":P", ";)", ":/", ":3", ":>", ":<",
    "ㅋ", "ㅎ", "ㅠ", "ㅜ", "ㄷ", "ㄱ",
    "ㅋㅋ", "ㅎㅎ", "ㅠㅠ", "ㅜㅜ", "ㄷㄷ", "ㄱㄱ",
    "ㅋㅋㅋ", "ㅎㅎㅎ", "ㅠㅠㅠ", "ㅜㅜㅜ",
    "ㅋㅋㅋㅋㅋ", "ㅎㅎㅎㅎㅎ", "ㅠㅠㅠㅠㅠ",
    "ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ", "ㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎ",
    
    # 특수문자 조합 (강화)
    "@@@", "###", "***", "!!!", "???", "~~~",
    "...", "....", ".....", "......",
    "!?", "?!", "!?!", "?!?", "!!??",
    "ㅋㅎ", "ㅎㅋ", "ㅋㅋㅎㅎ", "ㅎㅎㅋㅋ",
    "^^", "^^;", "^^;;", "^^;;;",
    
    # 띄어쓰기 포함 (강화)
    "^ ^", "^ _ ^", ". . .", "? ? ?", "! ! !",
    "ㅋ ㅋ ㅋ", "ㅎ ㅎ ㅎ", "ㅠ ㅠ ㅠ",
    "^ 0 ^", "^ - ^",
    
    # 멘션/태그 형식 (강화)
    "@user", "@님", "#태그", "#해시태그",
    "@all", "@everyone", "@here",
    "@admin", "@mod", "@팀장", "@님들",
    "#공지", "#이벤트", "#꿀팁", "#정보",
    
    # 숫자+특수문자 (강화)
    "1234", "12345", "123456", "1", "2", "3", "0",
    "100", "200", "300", "1000", "10000",
    "1!", "2@", "3#", "4$", "5%",
    "100%", "50%", "10%", "1위", "2위", "3위",
    
    # 짧은 정상 문자열 (강화)
    "ㅇㅇ", "ㅇㅋ", "ㄴㄴ", "ㄱㄱ", "ㄴㅇ", "ㅇㄴ",
    "ㅎㅇ", "ㅂㅂ", "ㄱㅅ", "ㅊㅋ", "ㄳ",
    "ㅇㅇㅇ", "ㄴㄴㄴ", "ㄱㄱㄱ",
    "ok", "OK", "ㅇㅋ", "오키", "오케이",
    "ㄱㄱㄱㄱ", "고고고고", "고고",
    "ㅇㅇㅇㅇ", "ㄴㄴㄴㄴ",
    
    # 게임/인터넷 은어 (정상)
    "gg", "GG", "wp", "WP", "glhf", "GLHF",
    "lol", "LOL", "lmao", "LMAO",
    "ㄹㅇ", "ㅇㅈ",
    "ez", "EZ", "ㅊㅊ", "ㅍㅌ",
    
    # 추가: 단순 반복 패턴
    "ㅋㅋㅋㅋㅋㅋ", "ㅎㅎㅎㅎㅎㅎ", "ㅜㅜㅜㅜㅜㅜ", "ㅠㅠㅠㅠㅠㅠ",
    "ㄷㄷㄷㄷ", "ㄱㄱㄱㄱ", "ㅇㅇㅇㅇㅇ",
    
    # 추가: 혼합 이모티콘
    "ㅋㅋ^^", "ㅎㅎ^^", "^^ㅋㅋ", "ㅠㅠ;;", "ㅜㅜ;;",
    "ㅋㅋㅋ??", "ㅎㅎㅎ!!", "??ㅋㅋ", "!!ㅎㅎ",
    
    # 추가: URL/이메일 패턴 (정상)
    "www", "http", "https", ".com", ".kr", ".net",
    
    # 추가: 시간/날짜 패턴
    "오전", "오후", "AM", "PM", "24시", "365일",
]

df_special = pd.DataFrame({
    'text': special_char_samples,
    'label': 0  # 정상
})

# 중복 제거
df_special = df_special[~df_special['text'].astype(str).str.lower().isin(existing_texts)]

print(f"✅ 특수문자 정상 데이터 추가: {len(df_special)}개 (E4보다 강화)")

In [ ]:
# ========================================
# 📊 4) 데이터 합치기
# ========================================
df = pd.concat([df_base, df_lol_aug, df_special], ignore_index=True)

# 셔플
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"\n📊 최종 데이터 분포:")
print(f"   총 데이터: {len(df)}개")
print(f"   - 정상(0): {len(df[df['label']==0])}개")
print(f"   - 악성(1): {len(df[df['label']==1])}개")
print(f"   - 악성:정상 비율: {len(df[df['label']==1])/len(df[df['label']==0]):.2f}:1")

print(f"\n📈 데이터 증가:")
print(f"   기존: {len(df_base)}개")
print(f"   LOL 욕설 추가: +{len(df_lol_aug)}개")
print(f"   특수문자 정상 추가: +{len(df_special)}개")
print(f"   최종: {len(df)}개")

In [ ]:
# ========================================
# 🔀 5) Train/Test 분할
# ========================================
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['label']
)
print(f"Train: {len(train_df)}, Test: {len(test_df)}")
print(f"Train 비율 - 정상: {len(train_df[train_df['label']==0])}, 악성: {len(train_df[train_df['label']==1])}")

In [ ]:
# ========================================
# 🔤 6) 토큰화
# ========================================
MODEL_ID = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def tokenize_fn(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

raw_ds = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(test_df),
})

tokenized = raw_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
tokenized = tokenized.rename_column("label", "labels")
tokenized.set_format("torch")

print("✅ 토큰화 완료!")

In [ ]:
# ========================================
# ⚖️ 7) 커스텀 가중치 설정
# ========================================
class_weights = torch.tensor([W0, W1], dtype=torch.float)
print(f"⭐ 커스텀 Class weights: {class_weights}")
print(f"   정상(0) 가중치: {W0} → FP 벌점 강화")
print(f"   악성(1) 가중치: {W1}")
print(f"   정상:악성 = {W0}:{W1} = {W0/W1:.0f}:1 (실제 데이터 비율 반영)")

In [ ]:
# ========================================
# 🏋️ 8) 모델 및 Trainer 설정
# ========================================
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds, zero_division=0)
    rec = recall_score(labels, preds, zero_division=0)
    f1 = f1_score(labels, preds, zero_division=0)
    
    cm = confusion_matrix(labels, preds)
    tn, fp, fn, tp = cm.ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    
    try:
        probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
        auc = roc_auc_score(labels, probs)
    except:
        auc = float("nan")
    
    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "roc_auc": auc,
        "FP": fp,
        "FN": fn,
        "FPR": fpr,
    }

base_model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID, num_labels=2)

# ⭐ 핵심: 커스텀 가중치 적용
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        # ⭐ 커스텀 가중치 사용
        loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

os.makedirs(OUTPUT_DIR, exist_ok=True)

args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = WeightedTrainer(
    model=base_model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("✅ Trainer 설정 완료!")
print(f"📌 E5 특징: E4 데이터 + class_weights=[{W0}, {W1}] (정상 4배)")

In [ ]:
# ========================================
# 🚀 9) 학습 실행
# ========================================
trainer.train()

In [ ]:
# ========================================
# 📊 10) 최종 평가
# ========================================
final_metrics = trainer.evaluate()

print("\n" + "="*60)
print(f"📊 E5 최종 평가 결과 (데이터 증강 + 가중치 {W0}:{W1})")
print("="*60)
for key, value in final_metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

In [ ]:
# ========================================
# 📝 11) 실험 결과 기록
# ========================================
print("\n" + "="*60)
print("📝 실험 결과 기록 (복사용)")
print("="*60)

# 변수 미리 포맷팅
fpr_str = f"{final_metrics['eval_FPR']:.4f}" if final_metrics.get('eval_FPR') is not None else 'N/A'
prec_str = f"{final_metrics['eval_precision']:.4f}" if final_metrics.get('eval_precision') is not None else 'N/A'
rec_str = f"{final_metrics['eval_recall']:.4f}" if final_metrics.get('eval_recall') is not None else 'N/A'
f1_str = f"{final_metrics['eval_f1']:.4f}" if final_metrics.get('eval_f1') is not None else 'N/A'
auc_str = f"{final_metrics['eval_roc_auc']:.4f}" if final_metrics.get('eval_roc_auc') is not None else 'N/A'

print(f"""
#### 실험 ID: E5
- 변경 사항:
  - 데이터 증강: LOL 욕설({len(df_lol_aug)}개) + 특수문자 정상({len(df_special)}개, 강화)
  - 총 데이터: {len(df)}개 (기존 {len(df_base)}개 → +{len(df)-len(df_base)}개)
  - class_weights: [{W0}, {W1}] (정상 {int(W0/W1)}배 중시)
  - metric_for_best_model: f1
- 데이터:
  - train/validation split: 0.8/0.2 (random_state=42, stratify=label)
  - max_length=128, batch_size=32, epochs=3
- Validation 성능:
  - FP(정상→악성): {final_metrics.get('eval_FP', 'N/A')}
  - FN(악성→정상): {final_metrics.get('eval_FN', 'N/A')}
  - FPR: {fpr_str}
  - Precision: {prec_str}
  - Recall: {rec_str}
  - F1: {f1_str}
  - AUC: {auc_str}
- 코멘트:
  - E4 대비 FP ↓ 예상 (정상 가중치 강화)
  - 실제 데이터 비율(4:1) 반영
""")

In [ ]:
# ========================================
# 💾 12) 모델 저장
# ========================================
SAVE_DIR = os.path.join(OUTPUT_DIR, "best_model")
os.makedirs(SAVE_DIR, exist_ok=True)
trainer.save_model(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

print(f"✅ 모델 저장 완료: {SAVE_DIR}")

---
## 📋 E4 vs E5 비교

| 항목 | E4 | E5 |
|------|-----|-----|
| 데이터 증강 | LOL 욕설 + 특수문자 정상 | 동일 (특수문자 강화) |
| class_weights | 기본 (1:1) | **[4, 1]** (정상 4배) |
| 예상 효과 | 악성 탐지↑ | **FP ↓** (정상 오분류 감소) |

---
## 📋 다음 단계

E5 결과 확인 후:
1. **Threshold 튜닝**: 0.7 기준으로 조정
2. **모델비교_테스트.ipynb**: E4, E5 비교 평가